# Enter missing gene and sample name and s or e for start or end gene, and get the nearest gene in the ref and the sample, and append it to the old .csv

In [38]:
import pandas as pd

In [39]:
missing_gene='FBgn0011655'
start_end ='e'   # put s or e for start or end
file='T29A'    #put the name of the file, without extension

In [41]:
df_ref= pd.read_csv('dmel-all-r6.31.gtf', sep='\t')

In [42]:
df_ref.columns=['seqname','source','feature','start','end','Xscore','strand','Xframe','attributes']

In [43]:
df_ref.drop(df_ref[df_ref.feature != 'gene'].index, inplace=True)

In [44]:
df_ref["gene id"] = [x.replace("gene_id","").replace('"',"").strip() for z in df_ref.attributes.str.split(";").tolist() for x in z if "gene_id" in x]

In [45]:
df_ref= df_ref.drop(['source','attributes','Xscore','Xframe'], axis=1)

In [46]:
df_ref = df_ref.drop(df_ref[~df_ref['seqname'].isin(['X', '3L','3R', '2L','2R','4'])].index)

In [47]:
df_ref.reset_index(drop=True, inplace=True)

In [48]:
df_ref

,seqname,feature,start,end,strand,gene id
0,X,gene,20025099,20025170,+,FBgn0052826
1,X,gene,20051294,20052519,+,FBgn0031085
2,X,gene,20094398,20095767,+,FBgn0062565
3,X,gene,20133579,20138878,+,FBgn0031088
4,X,gene,20141819,20143188,-,FBgn0041626
...,...,...,...,...,...,...
17629,2L,gene,860197,861806,+,FBgn0031289
17630,2L,gene,877302,878270,+,FBgn0002936
17631,2L,gene,898500,901316,+,FBgn0002563
17632,2L,gene,914086,917805,+,FBgn0046113


# Enter Missing Gene: ref

In [49]:
missing_gene_info = df_ref.loc[(df_ref['gene id'] == missing_gene)]
missing_gene_info


,seqname,feature,start,end,strand,gene id
7842,3R,gene,31611046,31615375,+,FBgn0011655


In [50]:
chrom = missing_gene_info['seqname'].item()

In [51]:
df_ref.sort_values(by='start', inplace=True)

In [52]:
df_ref

,seqname,feature,start,end,strand,gene id
13856,4,gene,879,5039,+,FBgn0267363
7175,4,gene,5829,11765,-,FBgn0052011
7194,2L,gene,7529,9484,+,FBgn0031208
579,2L,gene,9839,21376,-,FBgn0002121
7174,4,gene,12640,25054,-,FBgn0052010
...,...,...,...,...,...,...
14591,3R,gene,32023064,32023483,+,FBgn0286918
4974,3R,gene,32038330,32039753,+,FBgn0039887
4977,3R,gene,32045314,32051993,-,FBgn0040206
4975,3R,gene,32052060,32054974,+,FBgn0002780


In [53]:
if start_end =='s':
    missing_gene_coordinate = missing_gene_info['start'].item()
    nearest_gene_ref = df_ref[(df_ref.start > missing_gene_coordinate) & (df_ref.seqname == chrom)].iloc[0]
else:
    missing_gene_coordinate = missing_gene_info['end'].item()
    nearest_gene_ref = df_ref[(df_ref.end < missing_gene_coordinate) & (df_ref.seqname == chrom)].iloc[-1]


In [54]:
nearest_gene_ref

seqname             3R
feature           gene
start         31596642
end           31612664
strand               -
gene id    FBgn0039858
Name: 4938, dtype: object

In [55]:
gene_id_ref =nearest_gene_ref['gene id']
gene_seqname_ref =nearest_gene_ref['seqname']
gene_coordinate_ref1=nearest_gene_ref['start']
gene_coordinate_ref2=nearest_gene_ref['end']

# Nearest gene in sample: set up new file

In [56]:
df= pd.read_csv(f'{file}_liftoff', sep='\t', header=None)

In [57]:
df

,0,1,2,3,4,5,6,7,8
0,2L,Liftoff,gene,32969,34924,.,+,.,"gene_id ""FBgn0031208""; gene_symbol ""CR11023""; ..."
1,2L,Liftoff,exon,32969,33556,.,+,.,"gene_id ""FBgn0031208""; gene_symbol ""CR11023""; ..."
2,2L,Liftoff,exon,33633,34924,.,+,.,"gene_id ""FBgn0031208""; gene_symbol ""CR11023""; ..."
3,2L,Liftoff,pseudogene,32969,34924,.,+,.,"gene_id ""FBgn0031208""; gene_symbol ""CR11023""; ..."
4,2L,Liftoff,gene,35278,46813,.,-,.,"gene_id ""FBgn0002121""; gene_symbol ""l(2)gl""; c..."
...,...,...,...,...,...,...,...,...,...
532993,X,Liftoff,exon,22684349,22684452,.,-,.,"gene_id ""FBgn0259858""; gene_symbol ""Su(Ste):CR..."
532994,X,Liftoff,ncRNA,22684349,22684452,.,-,.,"gene_id ""FBgn0259858""; gene_symbol ""Su(Ste):CR..."
532995,X,Liftoff,gene,22689754,22689860,.,-,.,"gene_id ""FBgn0259852""; gene_symbol ""Su(Ste):CR..."
532996,X,Liftoff,exon,22689754,22689860,.,-,.,"gene_id ""FBgn0259852""; gene_symbol ""Su(Ste):CR..."


In [58]:
df.columns=['seqname','source','feature','start','end','Xscore','strand','Xframe','attributes']
df.drop(df[df.feature != 'gene'].index, inplace=True)
df["gene id"] = [x.replace("gene_id","").replace('"',"").strip() for z in df.attributes.str.split(";").tolist() for x in z if "gene_id" in x]
df = df.drop(['source','attributes','Xscore','Xframe'], axis=1)

In [59]:
df.sort_values(by='start', inplace=True)

In [60]:
if start_end =='s':
    nearest_gene = df.loc[(df['seqname'] == gene_seqname_ref)\
                          & (df.start > missing_gene_coordinate)].iloc[0]        
else:
    nearest_gene = df.loc[(df['seqname'] == gene_seqname_ref)\
                          & (df.end < missing_gene_coordinate)].iloc[-1] 

In [61]:
gene_id =nearest_gene['gene id']
gene_seqname =nearest_gene['seqname']
coordinate1=nearest_gene['start']
coordinate2=nearest_gene['end']

In [62]:
print(f'Missing gene:  {missing_gene} from sample {file} coordinate: {missing_gene_coordinate}')
print('-------------------------------------------------')
print(f'Reference nearest gene:   {gene_id_ref} at {gene_coordinate_ref1} : {gene_coordinate_ref1} \
on {gene_seqname_ref}')
print(f'Sample nearest gene:      {gene_id} at {coordinate1} : {coordinate2} on {gene_seqname}')
print('-------------------------------------------------')

Missing gene:  FBgn0011655 from sample T29A coordinate: 31615375
-------------------------------------------------
Reference nearest gene:   FBgn0039858 at 31596642 : 31596642 on 3R
Sample nearest gene:      FBgn0039709 at 31601509 : 31614566 on 3R
-------------------------------------------------


In [63]:
nearest_gene = df.loc[(df['gene id'] == gene_id)].drop(['strand','feature'], axis=1)


In [64]:
nearest_gene

,seqname,start,end,gene id
418221,3R,31601509,31614566,FBgn0039709


# Append new coordinates to the old boundaries file. 

In [ ]:
df_new= pd.read_csv(f'boundaries_{file}_chakra.csv', sep=',')

In [ ]:
df_new

In [ ]:
seqname = nearest_gene['seqname'].item()
start = nearest_gene['start'].item()
end = nearest_gene['end'].item()
gene_id = nearest_gene['gene id'].item()

In [ ]:
series=[seqname,start,end,gene_id]

In [ ]:
df_new.loc[-1] = series

In [ ]:
df_new= df_new.sort_values('seqname')

In [ ]:
df_new.to_csv(f'boundaries_{file}_chakra.csv', index=False)

In [65]:
gene = df.loc[(df['gene id'] == 'FBgn0283724')]
gene

,seqname,feature,start,end,strand,gene id
249235,3L,gene,10266298,10272657,-,FBgn0283724


In [66]:
13036082-10266298


2769784